# Political Affiliation and SNAP Usage of Oregon
In order to delve into possible factors for SNAP, we wanted to see if there was a correlation between political affiliation and SNAP Usage in Oregon, a state with high SNAP Usage.


## Importing Data

First I import a GeoJSON file that has all the eometry information for US counties, where the feature.id is a FIPS code, since the dataset I have uses FIPS code.

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties["features"][0]

Then, I import the 2020 election result by county data in .csv file, which is from a github that scraped data from the new york times.

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2020_US_County_Level_Presidential_Results.csv",
                   dtype={"fips": str})

## Cleaning up the Data
I want to clean up how the data set looks, like the columns, rows, as well as how the percentages are displayed.

Because the dataset has more information than needed, I trim it.

I check to see how the dataset looks like.

In [ ]:
df.head()

Then, I look at what columns there are.

In [ ]:
df.info()

I choose the columns I want to keep and rename them.

In [ ]:
df = df[['state_name', 'county_fips', 'county_name', 'votes_gop', 'votes_dem', 'total_votes','per_gop','per_dem']]

In [ ]:
df.columns = ['State Name','FIPS','County Name','GOP Votes','Democratic Votes', 'Total Votes','GOP Percentage','Democratic Percentage']

Then, I make sure to check if I have all the columns I wanted.

In [ ]:
df.info()

Because I'm only looking at Oregon, I create a dataframe with only Oregon as the State Name.

In [ ]:
df_oregon = df.loc[df['State Name'] == 'Oregon']
df_oregon.head(50)

## Data Visualizations
### Oregon's political affiliation
Since Oregon's overall SNAP usage is higher, I wanted to compare that with the overall political affiliaton of the state.

I totaled up the sum of both GOP and Democratic Votes and dividied it by the sum of Total votes for each, multiplied by 100 in order to get the overall percentage of GOP and Democratic Votes in Oregon.

In [ ]:
df_goptotal = (sum(df_oregon['GOP Votes']) / sum (df_oregon['Total Votes'])* 100)


In [ ]:
df_demtotal = (sum(df_oregon['Democratic Votes']) / sum (df_oregon ['Total Votes'])*100)


Then, I created a bar chart that showed the percentage of votes for each candidate in either GOP or democratic party.

In [ ]:
from matplotlib import pyplot as plt

plotdata = pd.DataFrame(
    {"affiliation": [df_goptotal, df_demtotal]}, 
    index=["Republican", "Democratic"])
plotdata.plot(kind="bar")

plt.xticks(rotation=0, horizontalalignment="center")
plt.title("Political Affiliation in Oregon")
plt.xlabel("Political Affiliation")
plt.ylabel("Percentage of Votes for party candidate")



We can see that democratic votes are higher than republican GOP votes in Oregon.

### Party Affiliation by County
Next we go into each county in Oregon to see if different counties and their political alignments have a correlation with their SNAP usage.

I make sure to import the libraries needed.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
print(gpd.datasets.available)

Then, I create a legend for my cloropleth map. 

In [ ]:
cat_dict = {
    0: {
        'color': 'republican',
        'legend': r'$GOP$'
    },
    1: {
        'color': 'democratic',
        'legend': r'$DEM$'
        
    },
}

I make sure that the color will match the affiliation of each county and create the map.

In [ ]:
df_oregon['Affiliation'] = None

In [ ]:
df_oregon.loc[(df_oregon['Democratic Percentage'] > df_oregon['GOP Percentage']), 'Affiliation'] = cat_dict[1]['color']
df_oregon.loc[(df_oregon['GOP Percentage'] > df_oregon['Democratic Percentage']), 'Affiliation'] = cat_dict[0]['color']

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = df_oregon

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations="FIPS", color="Affiliation",    
                    color_discrete_map={
                "republican": "red",
                "democratic": "blue",},
                           scope = "usa", hover_data=["County Name","Democratic Percentage", "GOP Percentage"], title = "Political Affiliation of counties in Oregon"
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout( margin={"r":0,"t":50,"l":0,"b":0})
fig.show()

### Comparision with SNAP Usage per County

I import my SNAP data for each county.

In [ ]:
snap = gpd.read_file('Data/acs2019_5yr_B22003_05000US41035.geojson')

In [ ]:
snap.head()

I clean up the data for that, removing columns I don't want and renaming my columns

In [ ]:
list(snap)

In [ ]:
columns_to_keep = ['geoid',
 'name',
 'B22003001',
 'B22003002',
 'B22003005',
 'geometry']

In [ ]:
snap = snap[columns_to_keep]

In [ ]:
list(snap)

In [ ]:
snap.columns = ['GeoID',
 'Name',
 'Total Households',
 'Household received Food Stamps/SNAP in the past 12 months',
 'Household did not receive Food Stamps/SNAP in the past 12 months',
 'geometry']

In [ ]:
snap.head()

I also edit my GeoID so that 0500US isn't there and only the FIPS of the county is. 

In [ ]:
snap['GeoID'] = snap['GeoID'].str.replace('05000US','')
snap.tail()

Then, I normalize my data by calculating the percent of househoulds that receive Food Stamps and SNAP since each county has a different amount of population.

In [ ]:
snap['Percent of Households that received Food Stamps/SNAP in the past 12 months'] = (snap['Household received Food Stamps/SNAP in the past 12 months'] /snap['Total Households']*100)


In [ ]:
snap.head()

In [ ]:

gdf = snap



fig = px.choropleth(gdf, geojson=counties, locations= "GeoID", 
                    color="Percent of Households that received Food Stamps/SNAP in the past 12 months",
                    color_continuous_scale="Greens",
                           scope = "usa", hover_data=["Name","Percent of Households that received Food Stamps/SNAP in the past 12 months"], 
                           title = "Percentage of Households with SNAP in Oregon by counties"
                          )


fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout( margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


There seems to be no correlation among the political affiliation within counties and their SNAP usage. However, because each county has a different number of people and therefore different percentage that uses SNAP, this may have lead to inconclusive results, as certain counties are more populated.